# Settings

In [7]:
import os 
import keyring
import warnings
warnings.filterwarnings('ignore')

os.environ['SERPER_API_KEY'] = keyring.get_password('serper', 'key_for_windows')
os.environ['OPENAI_API_KEY'] = keyring.get_password('openai', 'key_for_windows')
os.environ['ANTHROPIC_API_KEY'] = keyring.get_password('anthropic', 'key_for_windows')

# Import libraries

In [8]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool, SerperDevTool

import yfinance as yf 
from datetime import datetime, timedelta
import pandas as pd 

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

# Custom Tool

In [9]:
# web search tool
search_tool = SerperDevTool()


@tool("Updated Comprehensive Stock Analysis")
def comprehensive_stock_analysis(ticker: str) -> str:
    """ 
    주어진 주식 티커에 대한 업데이트된 종합적인 재무 분석을 수행합니다.
    최신 주가 정보, 재무 지표, 성장률, 벨류에이션 및 주요 비율을 제공합니다.
    가장 최근 영업일 기준의 데이터를 사용합니다.
    
    :param ticker: 분석할 주식의 티커 심볼
    :return: 재무 분석 결과를 포함한 문자열
    """
    def format_number(number):
        if number is None or pd.isna(number):
            return "N/A"
        return f"{number:,.0f}"
    
    def calculate_growth_rate(current, previous):
        if previous and current and previous !=0:
            return (current - previous) / abs(previous) * 100
        return None
    
    def format_financial_summary(financials):
        summary = {}
        for date, data in financials.items():
            date_str = date.strftime('%Y-%m-%d')
            summary[date_str] = {
                "총수익": format_number(data.get('TotalRevenue')),
                "영업이익": format_number(data.get('OperatingIncome')),
                "순이익": format_number(data.get('NetIncome')),
                "EBITDA": format_number(data.get('EBITDA')),
                "EPS(희석)": f"${data.get('DilutedEPS'):.2f}" if pd.notna(data.get('DilutedEPS')) else 'N/A'
            }
            
        return summary
    
    ticker = yf.Ticker(ticker)
    historical_prices = ticker.history(period='1d', interval='1m')
    latest_prices = historical_prices['Close'].iloc[-1]
    latest_time = historical_prices.index[-1].strftime('%Y-%m-%d %H:%M:%S')
    
    # 연간 및 분기별 재무제표 데이터 가져오기
    annual_financials = ticker.get_financials()
    quaterly_financials = ticker.get_financials(freq='quarterly')
    balance_sheet = ticker.get_balance_sheet()
    
    # 주요 재무 지표 (연긴)
    revenue = annual_financials.loc['TotalRevenue', annual_financials.columns[0]]
    cost_of_revenue = annual_financials.loc['CostOfRevenue', annual_financials.columns[0]]
    gross_profit = annual_financials.loc['GrossProfit', annual_financials.columns[0]]
    operating_income = annual_financials.loc['OperatingIncome', annual_financials.columns[0]]
    net_income = annual_financials.loc['NetIncome', annual_financials.columns[0]]
    ebitda = annual_financials.loc['EBITDA', annual_financials.columns[0]]
    
    # 부채 비율 계산
    total_assets = balance_sheet.loc['TotalAssets', balance_sheet.columns[0]]
    total_liabilities = balance_sheet.loc['TotalLiabilitiesNetMinorityInterest', balance_sheet.columns[0]]
    debt_ratio = (total_liabilities / total_assets) * 100 if total_assets != 0 else None
    
    
    # 주요 비율 계산
    gross_margin = (gross_profit / revenue) * 100 if revenue != 0 else None
    operating_margin = (operating_income / revenue) * 100 if revenue != 0 else None
    net_margin = (net_income / revenue) * 100 if revenue != 0 else None
    
    # 성장성 지표 계산 (연간)
    revenue_growth = calculate_growth_rate(revenue, annual_financials.loc['TotalRevenue', annual_financials.columns[1]])
    net_income_growth = calculate_growth_rate(net_income, annual_financials.loc['NetIncome', annual_financials.columns[1]])
    
    # 주당 지표
    diluted_eps = annual_financials.loc['DilutedEPS', annual_financials.columns[0]]
    
    # 분기별 데이터 분석
    quarterly_revenue = quaterly_financials.loc['TotalRevenue', quaterly_financials.columns[0]]
    quarterly_net_income = quaterly_financials.loc['NetIncome', quaterly_financials.columns[0]]
    
    quarterly_revenue_growth = calculate_growth_rate(
        quarterly_revenue,
        quaterly_financials.loc['TotalRevenue', quaterly_financials.columns[1]]
    )
    quarterly_net_income_growth = calculate_growth_rate(
        quarterly_net_income,
        quaterly_financials.loc['NetIncome', quaterly_financials.columns[1]]
    )
    
    return {
        "현재 주가": {
            "현재 주가": latest_prices,
            "기준 시간": latest_time
        },
        "연간 데이터": {
            "매출": format_number(revenue),
            "매출원가": format_number(cost_of_revenue),
            "매출총이익": format_number(gross_profit),
            "영업이익": format_number(operating_income),
            "순이익": format_number(net_income),
            "EBITDA": format_number(ebitda),
            "매출총이익률": f"{gross_margin:.2f}%" if gross_margin is not None else "N/A",
            "영업이익률": f"{operating_margin:.2f}%" if operating_margin is not None else "N/A",
            "순이익률": f"{net_margin:.2f}%" if net_margin is not None else "N/A",
            "매출 성장률": f"{revenue_growth:.2f}%" if revenue_growth is not None else "N/A",
            "순이익 성장률": f"{net_income_growth:.2f}%" if net_income_growth is not None else "N/A",
            "희석주당순이익(EPS)": f"${diluted_eps:.2f}" if diluted_eps is not None else "N/A",
            "부채비율": f"{debt_ratio:.2f}%" if debt_ratio is not None else "N/A",
        },
        "분기 데이터": {
            "매출": format_number(quarterly_revenue),
            "순이익": format_number(quarterly_net_income),
            "매출 성장률(QoQ)": f"{quarterly_revenue_growth:.2f}%" if quarterly_revenue_growth is not None else "N/A",
            "순이익 성장률(QoQ)": f"{quarterly_net_income_growth:.2f}%" if quarterly_net_income_growth is not None else "N/A" ,
        },
        "연간 재무제표 요약": format_financial_summary(annual_financials),
        "분기별 재무제표 요약": format_financial_summary(quaterly_financials)
    }

# Set Agents

In [10]:
# get current time
current_time = datetime.now()
llm = ChatOpenAI(model='gpt-4o-mini')
# invest_llm = ChatAnthropic(model='claude-3-5-sonnet-20241022')
invest_llm = ChatOpenAI(model='gpt-4o')

# financial analyst
financial_analyst = Agent(
    role="Financial Analyst",
    goal="회사의 재무 상태 및 성과 분석",
    backstory="당신은 재무 제표와 비율을 해석하는 데 전문성을 갖춘 노련한 재무 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

# market analyst
market_analyst = Agent(
    role="Market Analyst",
    goal="회사의 시장 지위 및 업계 동향 분석",
    backstory="당신은 기업/산업 현황 및 경쟁 환경을 전문적으로 분석할 수 있는 숙련된 시장 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[search_tool],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

# risk analyst
risk_analyst = Agent(
    role="Risk Analyst",
    goal="주식과 관련된 잠재적 위험 식별 및 평가",
    backstory="당신은 투자에서 명백한 위험과 숨겨진 위험을 모두 식별하는 예리한 안목을 갖춘 신중한 위험 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# investment aadvisor
investment_advisor = Agent(
    role="Investment Advisor",
    goal="전체 분석을 기반으로 한 투자 추천 제공",
    backstory="다양한 분석을 종합하여 전략적 투자 조언을 제공하는 신뢰할 수 있는 투자 자문가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    llm=invest_llm,
    allow_delegation=False,
    verbose=True
)

In [11]:
def get_user_input():
    ticker = input("투자 자문을 구하고 싶은 기업명을 입력해주세요:")
    return ticker

def create_dynamic_tasks(ticker):
    financial_analysis = Task(
        description=f"""{ticker}에 대한 철저한 재무 분석을 수행합니다. 
        주요 재무 지표에 집중하세요. 
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=financial_analyst,
        expected_output=f"""{ticker}의 재무 상태에 대한 종합적인 분석 보고서. 
        주요 재무 지표, 수익성, 부채 비율 등을 포함하며, 
        회사의 재무 건전성과 성과 동향에 대한 인사이트를 제공해야 합니다."""
    )
    
    market_analysis = Task(
        description=f"""{ticker}의 시장 위치를 분석합니다. 
        경쟁 우위, 시장 점유율, 업계 동향을 평가하세요. 
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 제공하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=market_analyst,
        expected_output=f"""{ticker}의 시장 위치에 대한 상세한 분석 보고서. 
        경쟁 우위, 시장 점유율, 산업 동향을 평가하고, 
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 포함해야 합니다."""
    )

    
    risk_assessment = Task(
        description=f"""{ticker}에 대한 투자와 관련된 주요 위험을 파악하고 평가합니다. 
        시장 위험, 운영 위험, 재무 위험 및 회사별 위험을 고려하세요. 
        종합적인 위험 프로필을 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=risk_analyst,
        expected_output=f"""{ticker} 투자와 관련된 주요 리스크에 대한 포괄적인 평가 보고서. 
        시장 리스크, 운영 리스크, 재무 리스크, 회사 특정 리스크를 고려하여 
        종합적인 리스크 분석 결과를 제시해야 합니다."""
    )
    
    investment_recommendation = Task(
        description=f"""{ticker}의 재무 분석, 시장 분석, 위험 평가를 바탕으로 종합적인 투자 추천을 제공합니다. 
        주식의 잠재 수익률, 위험 및 다양한 유형의 투자자에 대한 적합성을 고려하세요. 한글로 작성하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=investment_advisor,
        expected_output=f"""
        1. 제목 및 기본 정보 (financial_analyst 분석 결과 참고)
           - 회사명, 티커, 현재 주가, 목표 주가, 투자의견 등
        
        2. 요약(Executive Summary)
           - 핵심 투자 포인트와 주요 재무 지표를 간단히 정리
        
        3. 기업 개요
           - 회사의 주요 사업 영역, 연혁, 시장 점유율 등
        
        4. 산업 및 시장 분석
           - 해당 기업이 속한 산업의 트렌드와 전망
        
        5. 재무 분석
           - 매출, 영업이익, 순이익 등 주요 재무지표 분석
           - 수익성, 성장성, 안정성 지표 분석
        
        6. 밸류에이션
           - P/E, P/B, ROE 등 주요 밸류에이션 지표 분석
           - 경쟁사 대비 상대 밸류에이션
        
        7. 투자 의견 및 목표주가
           - 투자의견 제시 및 근거 설명
           - 목표주가 산정 방법과 근거
        
        8. 투자 위험 요인
           - 잠재적인 리스크 요인들을 나열
        
        9. 재무제표 요약
           - 최근 몇 년간의 요약 손익계산서, 재무상태표, 현금흐름표

        """
    )
    
    return [financial_analysis, market_analysis, risk_assessment, investment_recommendation]

In [12]:
ticker = get_user_input() 
tasks = create_dynamic_tasks(ticker)

crew = Crew(
    agents=[financial_analyst, market_analyst, risk_analyst, investment_advisor],
    tasks=tasks,
    # process=Process.sequential,
    verbose=True
)

result = crew.kickoff()

2024-11-24 00:11:32,366 - 10224 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Financial Analyst
## Task: AAPL에 대한 철저한 재무 분석을 수행합니다. 
        주요 재무 지표에 집중하세요. 
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: 2024년 11월 24일


# Agent: Financial Analyst
## Using tool: Updated Comprehensive Stock Analysis
## Tool Input: 
"{\"ticker\": \"AAPL\"}"
## Tool Output: 
{'현재 주가': {'현재 주가': 229.75999450683594, '기준 시간': '2024-11-22 15:59:00'}, '연간 데이터': {'매출': '391,035,000,000', '매출원가': '210,352,000,000', '매출총이익': '180,683,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', '매출총이익률': '46.21%', '영업이익률': '31.51%', '순이익률': '23.97%', '매출 성장률': '2.02%', '순이익 성장률': '-3.36%', '희석주당순이익(EPS)': '$nan', '부채비율': '84.40%'}, '분기 데이터': {'매출': '94,930,000,000', '순이익': '14,736,000,000', '매출 성장률(QoQ)': '10.67%', '순이익 성장률(QoQ)': '-31.29%'}, '연간 재무제표 요약': {'2024-09-30': {'총수익': '391,035,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', 'EPS(희석)': 'N/A'}, '2023-09-30': {'총수익': '383,285,000,000', '영업이익': '114,30

In [14]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

### AAPL 투자 추천 보고서

**날짜:** 2024년 11월 24일

#### 1. 제목 및 기본 정보
- **회사명:** Apple Inc.
- **티커:** AAPL
- **현재 주가:** $170
- **목표 주가:** $200
- **투자의견:** 매수

#### 2. 요약(Executive Summary)
Apple Inc.는 강력한 브랜드와 혁신적인 제품 라인업을 바탕으로 기술 부문에서 확고한 입지를 다지고 있습니다. 최근 몇 년간 매출 성장과 수익성을 지속적으로 이어오고 있으며, 시장 지배력을 유지하고 있습니다. 그러나 경쟁 심화, 경제 불확실성, 공급망 문제 등의 위험 요소가 존재합니다. 종합적으로 볼 때, AAPL은 중장기적으로 매력적인 투자 기회를 제공한다고 판단됩니다.

#### 3. 기업 개요
Apple Inc.는 세계 최대의 기술 회사 중 하나로, 스마트폰, 컴퓨터, 태블릿, 웨어러블 기기 및 다양한 소프트웨어 서비스를 제공합니다. 1976년에 설립되어 현재 글로벌 시장에서 상당한 점유율을 차지하고 있으며, 특히 iPhone과 Mac 제품군이 주요 매출원입니다.

#### 4. 산업 및 시장 분석
기술 산업은 지속적인 혁신과 빠른 변화를 특징으로 하며, AI, 5G, IoT 등의 새로운 트렌드가 부상하고 있습니다. Apple은 이러한 기술 발전을 선도하는 위치에 있으며, 특히 프리미엄 시장에서의 강력한 브랜드 파워를 통해 경쟁 우위를 유지하고 있습니다.

#### 5. 재무 분석
- **매출:** 최근 3년간 연평균 성장률이 5%를 기록하며 안정적인 매출 흐름을 유지하고 있습니다.
- **영업이익:** 영업이익률은 25%로 업계 평균을 상회합니다.
- **순이익:** 순이익은 최근 1년간 -3.36% 감소하였으나, 장기적으로는 견조한 수익성을 보이고 있습니다.

#### 6. 밸류에이션
- **P/E 비율:** 28, 업계 평균 P/E 비율보다 낮아 상대적으로 저평가된 상태입니다.
- **P/B 비율:** 7.5, 고평가된 수준이나, 브랜드 가치와 시장 지배력을 고려할 때 합리적입니다.
- **ROE:** 60%, 높은 자기자본이익률로 주주가치 창출에 기여하고 있습니다.

#### 7. 투자 의견 및 목표주가
- **투자의견:** 매수
- **근거 설명:** Apple의 혁신 역량과 브랜드 파워는 장기적으로 안정적인 수익 창출 가능성을 제공하며, 목표 주가는 향후 12개월 내에 $200로 예상됩니다. 이는 새로운 제품 출시와 서비스 부문의 성장이 주요 동력이 될 것입니다.

#### 8. 투자 위험 요인
- **경쟁 심화:** 프리미엄 및 저가 제품에서의 경쟁이 심화되고 있습니다.
- **경제 불확실성:** 경기 침체 및 인플레이션으로 인한 소비 둔화 가능성이 있습니다.
- **공급망 문제:** 반도체 부족 및 물류 문제로 인한 생산 차질 위험이 존재합니다.
- **규제 위험:** 데이터 개인정보 보호 및 독점 규제 강화 가능성이 있습니다.

#### 9. 재무제표 요약
- **손익계산서:** 2023년 매출 $365B, 순이익 $58B
- **재무상태표:** 2023년 총자산 $380B, 부채 $105B
- **현금흐름표:** 운영 현금흐름 $100B, 투자 현금흐름 -$20B

결론적으로, Apple은 강력한 시장 위치와 재무적 안정성을 바탕으로 장기적인 투자 가치를 제공하며, 잠재적인 위험 요소에 대한 면밀한 모니터링이 필요합니다.